In [1]:
# SOURCE: https://github.com/anyscale/academy/blob/5046fdb4d94fcd7b9be6fa44e877283d401345c7/ray-crash-course/01-Ray-Tasks.ipynb

import time
import ray

In [2]:
ray.init(address='auto', ignore_reinit_error=True)
print(f'Dashboard URL: http://{ray.get_dashboard_url()}')

2021-05-24 22:08:02,598	INFO worker.py:640 -- Connecting to existing Ray cluster at address: 192.168.178.38:6379


Dashboard URL: http://127.0.0.1:8265


In [3]:
class Counter:
    """Remember how many times ``next()`` has been called."""
    def __init__(self, pause):
        self.count = 0
        self.pause = pause
    def next(self):
        time.sleep(self.pause)
        self.count += 1
        return self.count

In [8]:
@ray.remote
class RayCounter(Counter):
    """Remember how many times ``next()`` has been called."""
    def __init__(self, pause):
        super().__init__(pause)

    def get_count(self):
        return self.count

In [4]:
def counter_trial(count_to, num_counters = 1, pause = 0.01):
    print('not ray: count_to = {:5d}, num counters = {:4d}, pause = {:5.3f}: '.format(count_to, num_counters, pause), end='')
    start = time.time()
    counters = [Counter(pause) for _ in range(num_counters)]
    for i in range(num_counters):
        for n in range(count_to):
            counters[i].next()
    duration = time.time() - start
    print('time = {:9.5f} seconds'.format(duration))
    return count_to, num_counters, pause, duration

In [5]:
def ray_counter_trial(count_to, num_counters = 1, pause = 0.01):
    print('ray:     count_to = {:5d}, num counters = {:4d}, pause = {:5.3f}: '.format(count_to, num_counters, pause), end='')
    start = time.time()
    final_count_futures = []
    counters = [RayCounter.remote(pause) for _ in range(num_counters)]
    for i in range(num_counters):
        for n in range(count_to):
            counters[i].next.remote()
        final_count_futures.append(counters[i].get_count.remote())
    ray.get(final_count_futures)  # Discard result, but wait until finished!
    duration = time.time() - start
    print('time = {:9.5f} seconds'.format(duration))
    return count_to, num_counters, pause, duration

In [6]:
count_to = 10
for num_counters in [1, 2, 3, 4]:
    counter_trial(count_to, num_counters, 0.0)
for num_counters in [1, 2, 3, 4]:
    counter_trial(count_to, num_counters, 0.1)
for num_counters in [1, 2, 3, 4]:
    counter_trial(count_to, num_counters, 0.2)

not ray: count_to =    10, num counters =    1, pause = 0.000: time =   0.00018 seconds
not ray: count_to =    10, num counters =    2, pause = 0.000: time =   0.00036 seconds
not ray: count_to =    10, num counters =    3, pause = 0.000: time =   0.00018 seconds
not ray: count_to =    10, num counters =    4, pause = 0.000: time =   0.00016 seconds
not ray: count_to =    10, num counters =    1, pause = 0.100: time =   1.00140 seconds
not ray: count_to =    10, num counters =    2, pause = 0.100: time =   2.00253 seconds
not ray: count_to =    10, num counters =    3, pause = 0.100: time =   3.00437 seconds
not ray: count_to =    10, num counters =    4, pause = 0.100: time =   4.00565 seconds
not ray: count_to =    10, num counters =    1, pause = 0.200: time =   2.00239 seconds
not ray: count_to =    10, num counters =    2, pause = 0.200: time =   4.00467 seconds
not ray: count_to =    10, num counters =    3, pause = 0.200: time =   6.00670 seconds
not ray: count_to =    10, num c

In [9]:

count_to = 10
for num_counters in [1, 2, 3, 4]:
    ray_counter_trial(count_to, num_counters, 0.0)
for num_counters in [1, 2, 3, 4]:
    ray_counter_trial(count_to, num_counters, 0.1)
for num_counters in [1, 2, 3, 4]:
    ray_counter_trial(count_to, num_counters, 0.2)

ray:     count_to =    10, num counters =    1, pause = 0.000: time =   0.65209 seconds
ray:     count_to =    10, num counters =    2, pause = 0.000: time =   0.55689 seconds
ray:     count_to =    10, num counters =    3, pause = 0.000: time =   0.51743 seconds
ray:     count_to =    10, num counters =    4, pause = 0.000: time =   0.11320 seconds
ray:     count_to =    10, num counters =    1, pause = 0.100: time =   1.03965 seconds
ray:     count_to =    10, num counters =    2, pause = 0.100: time =   1.06077 seconds
ray:     count_to =    10, num counters =    3, pause = 0.100: time =   1.01750 seconds
ray:     count_to =    10, num counters =    4, pause = 0.100: time =   1.58313 seconds
ray:     count_to =    10, num counters =    1, pause = 0.200: time =   2.01948 seconds
ray:     count_to =    10, num counters =    2, pause = 0.200: time =   2.06394 seconds
ray:     count_to =    10, num counters =    3, pause = 0.200: time =   2.49408 seconds
ray:     count_to =    10, num c

In [10]:
count_to=10
num_counters = 1
pauses=[]
durations=[]
ray_durations=[]
for pause in range(0,11):
    pauses.append(pause*0.01)
    _, _, _, duration = counter_trial(count_to, num_counters, pause*0.01)
    durations.append(duration)
    _, _, _, duration = ray_counter_trial(count_to, num_counters, pause*0.01)
    ray_durations.append(duration)

not ray: count_to =    10, num counters =    1, pause = 0.000: time =   0.00021 seconds
ray:     count_to =    10, num counters =    1, pause = 0.000: time =   0.04314 seconds
not ray: count_to =    10, num counters =    1, pause = 0.010: time =   0.10098 seconds
ray:     count_to =    10, num counters =    1, pause = 0.010: time =   0.15280 seconds
not ray: count_to =    10, num counters =    1, pause = 0.020: time =   0.20110 seconds
ray:     count_to =    10, num counters =    1, pause = 0.020: time =   0.22101 seconds
not ray: count_to =    10, num counters =    1, pause = 0.030: time =   0.30089 seconds
ray:     count_to =    10, num counters =    1, pause = 0.030: time =   0.31433 seconds
not ray: count_to =    10, num counters =    1, pause = 0.040: time =   0.40093 seconds
ray:     count_to =    10, num counters =    1, pause = 0.040: time =   0.41368 seconds
not ray: count_to =    10, num counters =    1, pause = 0.050: time =   0.50093 seconds
ray:     count_to =    10, num c

In [11]:
from bokeh_util import two_lines_plot  # utility we used in the previous lesson
from bokeh.plotting import show, figure
from bokeh.layouts import gridplot

Loading BokehJS ...

In [12]:
two_lines = two_lines_plot(
    "Pause vs. Execution Times (Smaller Is Better)", 'Pause', 'Time', 'No Ray', 'Ray',
    pauses, durations, pauses, ray_durations,
    x_axis_type='linear', y_axis_type='linear')
show(two_lines, plot_width=800, plot_height=400)

In [22]:
ray.shutdown()